### Libraries

In [399]:
from pathlib import Path
from pprint import pprint

### Variables

In [400]:
edl_path = 'reference\LWABDR60H.edl'
drop_frame = False
video_edits = []
audio_edits = []
# edit_files = []

### Open File

In [401]:
# Opens EDL and extracts its contents into a single string.
def open_edl(edl_path):
    with open(edl_path, 'r') as edl_file:
        edl_contents = edl_file.read()
    edl_extract(edl_contents)    

In [402]:
# Breasks EDL into a list of fields for each line
def edl_extract(edl_contents): 
    edl_lines = edl_contents.splitlines()

    edl_extracted = []

    for line in edl_lines:
        columns = line.split()
        edl_extracted.append(columns)
    
    
    simplify_edl(edl_extracted)


### Simplify EDL

In [403]:
def simplify_edl(edl):
	detect_dropframe(edl)
	simplified_edl = []
	for line in range(0, len(edl)):
		edl_line = []

		if len(edl[line]) > 3:
			if edl[line][-1] != edl[line][-2]:  #if this doesn't have duplicate timecode for edits
				edl_line.append(edl[line][2][0])  #video or audio
				edl_line.append(edl[line][3])  #cut or dissolve

				if edl[line][3] == "C":
					edl_line.append("0")  # dissolve length 0 for cuts
				else:
					edl_line.append(edl[line][4]) # dissolve length

				edl_line.append(edl[line][-2])  # in point
				edl_line.append(edl[line][-1])  # out point

				simplified_edl.append(edl_line)  #  adds edl line to the simplified EDL


		elif edl[line][0] == "*":  # lines with clip note
			edl_line.append(edl[line][0]) 
			edl_line.append(edl[line][1])

			simplified_edl.append(edl_line)  #  adds edl line to the simplified EDL 
	
	# pprint(simplified_edl)

	edl_split_video_audio(simplified_edl)	

### Detect Dropframe

In [404]:
def detect_dropframe(edl):
	global drop_frame
     
	if edl[1][0] == "FCM:":
            if edl[1][1] == "DROP":
                drop_frame = True
            else:
                drop_frame = False


### Split Video and Audio Edits

In [405]:
#  Looks for relevant edit information:  Drop/Non-Drop Frame, Title Name, Edits
def edl_split_video_audio(edl):
    # Clears the edit list for parsing
    global video_edits
    global audio_edits

    edl_video = []
    edl_audio = []

    for line in range(0, len(edl)):
        #  Splits audio and video edits into seperate EDLs
        if edl[line][0] == "V":
            # print("video")
            edl_video.append(edl[line])
            if line+1 < len(edl): 
                if edl[line+1][0] == "*":
                    edl_video.append(edl[line+1])                

        if edl[line][0] == "A":
            # print("audio")
            edl_audio.append(edl[line])
            if line+1 < len(edl):
                if edl[line+1][0] == "*":
                    edl_audio.append(edl[line+1])


    
    # Parses the edits
    # if edl_video:
    #     video_edits = parse_edits(edl_video)

    if edl_audio:
        audio_edits = parse_edits(edl_audio)

    # print("=========VIDEO=============")
    # pprint(edl_video)
    # print("=========AUDIO=============")
    pprint(edl_audio)


### Parse Edits

In [406]:
def parse_edits(edl):
    
    edit_list = []
    for line in range(0, len(edl)):  
        edits = {}     

        # print(edl[line][0])
        
        if edl[line][0] == "*":  

            #  Audio - Cut In - Cut Out
            if (edl[line-1][0] == "A"):
                edits = {
                    "dissolve_in" : 0,
                    "dissolve_out" : 0,
                    "edit_in" : edl[line-1][3],
                    "edit_out" : edl[line-1][4],
                    "edit_file" : edl[line][1],            
                }
                edit_list.append(edits)        

            # Dissolve in - Dissolve Out
            # elif  (edl[line-1][2][0] == "V"):
            #     print(edl[line-1][0])
   
                # edits = {
                #     "dissolve_in" : int(edl[line-2][4]),
                #     "dissolve_out" : int(edl[line+2][4]),
                #     "edit_in" : edl[line-2][7],
                #     "edit_out" : edl[line+2][8],
                #     "edit_file" : f"{edl[line][1]}.nbtitle",            
                # }
                # edit_list.append(edits)
            
            # # Dissolve in - Dissolve Out
            # elif  (line+3 <= len(edl) and 
            #        edl[line][1] == edl[line+3][1] and 
            #        edl[line-2][3] == "D" and 
            #        edl[line-1][3] == "C" and 
            #        edl[line+2][3] == "D"):
   
            #     edits = {
            #         "dissolve_in" : int(edl[line-2][4]),
            #         "dissolve_out" : int(edl[line+2][4]),
            #         "edit_in" : edl[line-2][7],
            #         "edit_out" : edl[line+2][8],
            #         "edit_file" : f"{edl[line][1]}.nbtitle",            
            #     }
            #     edit_list.append(edits)                
            
            # # Dissolve in - Cut out
            # elif (edl[line+1][5] != "00:00:00:00" and 
            #       edl[line-2][3] == "D" and 
            #       edl[line-1][3] == "C"):
            #     edits = {
            #         "dissolve_in" : int(edl[line-2][4]),
            #         "dissolve_out" : 0,
            #         "edit_in" : edl[line-2][7],
            #         "edit_out" : edl[line-1][7],
            #         "edit_file" : f"{edl[line][1]}.nbtitle",            
            #     }
            #     edit_list.append(edits)

            # # # Cut in - Dissolve Out
            # elif (line+3 <= len(edl) and 
            #       edl[line][1] == edl[line+3][1] and 
            #       edl[line-2][3] == "C" and edl[line-1][3] == "C" and 
            #       edl[line+2][3] == "D"):
            #     edits = {
            #         "dissolve_in" : 0,
            #         "dissolve_out" : int(edl[line+2][4]),
            #         "edit_in" : edl[line-1][7],
            #         "edit_out" : edl[line+2][8],
            #         "edit_file" : f"{edl[line][1]}.nbtitle",            
            #     }
            #     edit_list.append(edits)

            # #  Cut In - Cut Out
            # elif (line+1 <= len(edl) and 
            #       edl[line-2][3] == "C" and 
            #       edl[line-1][3] == "C" and 
            #       edl[line+1][3] == "C"):
            #     edits = {
            #         "dissolve_in" : 0,
            #         "dissolve_out" : 0,
            #         "edit_in" : edl[line-1][6],
            #         "edit_out" : edl[line-1][7],
            #         "edit_file" : f"{edl[line][1]}.nbtitle",            
            #     }
            #     edit_list.append(edits)

    return edit_list

### Execution 

In [407]:
open_edl(edl_path)
# print("------Video Edits-------")
# pprint(video_edits)
print("------Audio Edits-------")
pprint(audio_edits)

[['A', 'C', '0', '01:00:56:01', '01:00:58:27'], ['*', '{{$AUDIO}}']]
------Audio Edits-------
[{'dissolve_in': 0,
  'dissolve_out': 0,
  'edit_file': '{{$AUDIO}}',
  'edit_in': '01:00:56:01',
  'edit_out': '01:00:58:27'}]
